<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [71]:
import pandas as pd
import selenium
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import time
from selenium.webdriver.common.action_chains import ActionChains
import random as rnd
from tqdm.notebook import tqdm
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.relative_locator import locate_with

In [72]:
url = 'https://tabiturient.ru/slivlist/'

In [73]:
xpath_choose_vuz = '/html/body/div[1]/div[2]/div[1]/div/div[1]/div/table/tbody/tr/td[1]/table[1]/tbody/tr/td[2]'

In [74]:
month_dict = {'января': '01',
             'февраля': '02',
'марта': '03',
'апреля': '04',
'мая': '05',
'июня': '06',
'июля': '07',
'августа': '08',
'сентября': '09',
'октября': '10',
'ноября': '11',
'декабря': '12'
}

In [75]:
def get_uni_names_with_selenium(url: str):
    ser_uni = []
    ser_dates = []
    ser_advise = []

    options = webdriver.FirefoxOptions()
    options.set_preference("general.useragent.override", "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36")

    try:
        driver = webdriver.Firefox(
            executable_path="C:/Users/Iljam/PycharmProjects/parser_python_/geckodriver.exe",
            options=options
        )
        driver.get(url=url)
        driver.maximize_window()
        choose_vuz  = driver.find_element(by='xpath', value = xpath_choose_vuz)
        choose_vuz.click()

        with open("index_selenium.html", "w", encoding="utf-8") as file:
            file.write(driver.page_source)

        with open("index_selenium.html", encoding="utf-8") as file:
            src = file.read()

        soup = BeautifulSoup(src, "lxml")
        time.sleep(1)
        popup = soup.find_all('td', class_ = 'p10lm')
        for uni in popup:
            ser_uni.append(uni.b.text)
        return ser_uni

    except Exception as ex:
        print(ex)

    finally:
        driver.close()
        driver.quit()

In [6]:
def get_data_with_selenium(url: str,list_uni : list,  excel_file_name: str):
    ser_dates = []
    ser_advise = []
    df_list = []
    options = webdriver.FirefoxOptions()
    options.set_preference("general.useragent.override", "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36")
    df_total = pd.DataFrame(columns = ['Отзыв','День','Месяц','Год','Университет'])
    for name in tqdm(list_uni):
        time.sleep(rnd.randint(1,2))
        try:
            driver = webdriver.Firefox(
                executable_path="C:/Users/Iljam/PycharmProjects/parser_python_/geckodriver.exe",
                options=options
            )
            driver.get(url=url)
            driver.maximize_window()
            choose_vuz  = driver.find_element(by='xpath', value = xpath_choose_vuz)
            choose_vuz.click()

            search_input = driver.find_element(by='xpath', value = xpath_search)
            search_input.click()
            time.sleep(2)
            search_input.send_keys(name)
            time.sleep(2)
            search_input.send_keys(Keys.ENTER)
            time.sleep(3)
            result = driver.find_elements(By.CLASS_NAME, 'img5')[0]
            result.click()
            time.sleep(3)

            #Заставляем селениум скроллить окно, чтобы потом подгрузить данные
            for scroller in tqdm(range(100)):
                print('Scrolling page')
                for i in range(5):
                    driver.execute_script("window.scrollBy(0 , 100000);", "")
                print('Clicked')
                try:
                    download_window = driver.find_element(By.CLASS_NAME, 'mobpadd10 ')
                    download_window.click()
                except Exception:
                    break
                time.sleep(rnd.randint(2,4))

            time.sleep(rnd.randint(1,3))

            with open("index_selenium_uni.html", "w", encoding="utf-8") as file:
                file.write(driver.page_source)

            with open("index_selenium_uni.html", encoding="utf-8") as file:
                src_uni = file.read()

            soup = BeautifulSoup(src_uni, "lxml")
            time.sleep(3)
            advises = soup.find_all('div', style = 'text-align:justify;', class_ = 'font2')
            print('Parsing advises')

            for advise in advises:
                ser_advise.append(advise.text)

            days = []
            months = []
            years = []
            uni = []
            print('Parsing dates')
            for i in tqdm(range(len(ser_advise))):
                try:
                    date = driver.find_element(by = 'xpath', value = f'//*[@id="resultsliv"]/div[{i+1}]/div[1]/div[1]/div/div[1]/table/tbody/tr/td[2]/table/tbody/tr/td[5]/span[2]').text.split(' ')
                    days.append(date[0])
                    months.append(month_dict[date[1]])
                    years.append(date[2])
                    uni.append(name)
                except Exception:
                    pass
            print('Writing')
            df = pd.DataFrame(zip(ser_advise, days, months, years, uni), columns = ['Отзыв','День','Месяц','Год','Университет'])
            with pd.ExcelWriter(excel_file_name) as writer:
                df.to_excel(writer, sheet_name=f'DataFrame_{name}')
            df_list.append(df)

        except Exception as ex:
             print(ex)
        finally:
            driver.close()
            driver.quit()
    return df_list

In [76]:
ser_uni = get_uni_names_with_selenium(url)

C:\Users\Iljam\AppData\Local\Temp\ipykernel_8104\2796171620.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(


Message: Process unexpectedly closed with status 0



UnboundLocalError: local variable 'driver' referenced before assignment

In [10]:
ser_uni

['АГАТУ',
 'АГГПУ им. В.М. Шукшина',
 'АГИК',
 'АГИКИ',
 'АГМУ',
 'АГНИ',
 'АГПУ',
 'АГТУ',
 'АГУ',
 'АГУ',
 'АСОУ',
 'АлтГПУ',
 'АлтГТУ им. И.И. Ползунова',
 'АлтГУ',
 'Алтайский ГАУ',
 'Алтайский филиал РАНХиГС',
 'Алтайский филиал ФУ при Правительстве РФ',
 'Алфёровский университет',
 'АмГМА',
 'АмГПГУ',
 'АнГТУ',
 'Анапский филиал МПГУ',
 'Арзамасский филиал ННГУ',
 'Астраханский ГМУ',
 'Астраханский филиал РАНХиГС',
 'БАГСУ',
 'БГАУ',
 'БГИИК',
 'БГИТУ',
 'БГМУ',
 'БГПУ',
 'БГПУ им. М. Акмуллы',
 'БГТУ',
 'БГТУ им. В.Г. Шухова',
 'БГТУ им. Д. Ф. Устинова (Военмех)',
 'БГУ ',
 'БГУ',
 'БГУ им. Доржи Банзарова',
 'БИТИ НИЯУ МИФИ',
 'БФ ПНИПУ',
 'БФУ им. И. Канта',
 'Балаковский филиал РАНХиГС',
 'Балахнинский филиал ННГУ',
 'БашГАУ',
 'БашГУ',
 'Башкирский институт технологий и управления МГУТУ',
 'БелГУ',
 'Белгородский ГАУ',
 'БрГУ',
 'Брянский филиал РАНХиГС',
 'Брянский филиал РЭУ им. Плеханова',
 'Бурятская ГСХА',
 'ВАВТ Минэкономразвития России',
 'ВВГУ',
 'ВВФ МТУСИ',
 'ВГАУ 

In [12]:
list_data = get_data_with_selenium(url,list_uni, "Educational_parder.xlsx")

  0%|          | 0/167 [00:00<?, ?it/s]

C:\Users\Iljam\AppData\Local\Temp\ipykernel_12212\670770284.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(


  0%|          | 0/100 [00:00<?, ?it/s]

Scrolling page
Clicked
Parsing advises
Parsing dates


  0%|          | 0/1 [00:00<?, ?it/s]

...

C:\Users\Iljam\AppData\Local\Temp\ipykernel_12212\670770284.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(


  0%|          | 0/100 [00:00<?, ?it/s]

Scrolling page
Clicked
Parsing advises
Parsing dates


  0%|          | 0/1701 [00:00<?, ?it/s]

Writing


In [7]:
xpath_search = '//*[@id="popupcontent1"]/div[1]/div/table/tbody/tr/td[1]/input'

In [8]:
post_content = {'tag' : 'div',
               'class' : 'mobpadd20-2'}

In [25]:
non_na = []

In [26]:
for i in range(len(list_data)):
    if len(list_data[i]) !=0:
        non_na.append(i)

In [27]:
df_total = pd.DataFrame(list_data[non_na[0]], columns = [list_data[non_na[0]].columns])

In [45]:
ads = []
days = []
months = []
years = []
uni = []

for i in range(len(non_na)):
    for row in range(len(list_data[non_na[i]])):
        ads.append(list_data[non_na[i]].loc[row]['Отзыв'])
        days.append(list_data[non_na[i]].loc[row]['День'])
        months.append(list_data[non_na[i]].loc[row]['Месяц'])
        years.append(list_data[non_na[i]].loc[row]['Год'])
        uni.append(list_data[non_na[i]].loc[row]['Университет'])

In [48]:
df_total = pd.DataFrame(zip(ads, days,months,years,uni),  columns = list_data[non_na[0]].columns)

In [49]:

df_total.index = [i for i in range(len(df_total))]

In [50]:
df_total

,Отзыв,День,Месяц,Год,Университет
0,\n\n\nИнформации о этом ВУЗе в интернете нет (...,21,08,2021,АГУ
1,\n\n\nИнформации о этом ВУЗе в интернете нет (...,12,02,2017,АлтГУ
2,\n\n\nИнформации о этом ВУЗе в интернете нет (...,02,08,2022,Алфёровский университет
3,\n\n\nЗдравствуйте! Меня зовут Александр. Я жи...,01,11,2016,Алфёровский университет
4,\n\n\nИнформации о этом ВУЗе в интернете нет (...,08,06,2022,АмГМА
...,...,...,...,...,...
1696,"\n\n\nЗа 5 лет повидала многого, но по порядку...",08,11,2017,ЮФУ
1697,\n\n\nИнформации о этом ВУЗе в интернете нет (...,23,06,2021,ЯГПУ им. Ушинского
1698,\n\n\nЗдравствуйте! Меня зовут Александр. Я жи...,01,03,2021,ЯГПУ им. Ушинского
1699,"\n\n\nС удивлением для себя обнаружил, что про...",16,02,2021,ЯГПУ им. Ушинского


In [51]:
df_total.to_excel('Univercities.xlsx')

In [10]:
df_uni = pd.read_excel('Univercities.xlsx')

In [11]:
list_uni = list(df_uni['Университет'].unique())

In [20]:
print(list_data[non_na[20]])

                                               Отзыв День Месяц   Год  \
0  \n\n\nИнформации о этом ВУЗе в интернете нет (...   15    07  2022   

                        Университет  
0  ГМПИ им. М.М. Ипполитова-Иванова  


In [39]:
df_total.append(list_data[non_na[2]].loc[2])

KeyError: 2

In [33]:
pd.concat([df_total, list_data[non_na[2]]])

ValueError: no types given

In [44]:
list_data[non_na[2]].loc[0]

Отзыв          \n\n\nИнформации о этом ВУЗе в интернете нет (...
День                                                          02
Месяц                                                         08
Год                                                         2022
Университет                              Алфёровский университет
Name: 0, dtype: object

In [52]:
df_uni = pd.read_excel('Univercities.xlsx')

In [54]:
df_uni = df_uni.drop('Unnamed: 0', axis = 1)
df_uni

,Отзыв,День,Месяц,Год,Университет
0,\n\n\nИнформации о этом ВУЗе в интернете нет (...,21,8,2021,АГУ
1,\n\n\nИнформации о этом ВУЗе в интернете нет (...,12,2,2017,АлтГУ
2,\n\n\nИнформации о этом ВУЗе в интернете нет (...,2,8,2022,Алфёровский университет
3,\n\n\nЗдравствуйте! Меня зовут Александр. Я жи...,1,11,2016,Алфёровский университет
4,\n\n\nИнформации о этом ВУЗе в интернете нет (...,8,6,2022,АмГМА
...,...,...,...,...,...
1696,"\n\n\nЗа 5 лет повидала многого, но по порядку...",8,11,2017,ЮФУ
1697,\n\n\nИнформации о этом ВУЗе в интернете нет (...,23,6,2021,ЯГПУ им. Ушинского
1698,\n\n\nЗдравствуйте! Меня зовут Александр. Я жи...,1,3,2021,ЯГПУ им. Ушинского
1699,"\n\n\nС удивлением для себя обнаружил, что про...",16,2,2021,ЯГПУ им. Ушинского


In [66]:
import nltk
import re

In [71]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Iljam\AppData\Roaming\nltk_data...


True

In [63]:
for text in tqdm(df_uni['Отзыв']):
    tokens_list = nltk.word_tokenize(text)
    tokens_lsit_1 = []

  0%|          | 0/1701 [00:00<?, ?it/s]

In [64]:
for token in tqdm(tokens_list):
    tokens_lsit_1.append(token.lower())

  0%|          | 0/424 [00:00<?, ?it/s]

In [67]:
token_list_2 = []
for token in tqdm(tokens_lsit_1):
    if re.search('\w', token ) != None:
        token_list_2.append(token)

  0%|          | 0/424 [00:00<?, ?it/s]

In [72]:
from nltk.stem import WordNetLemmatizer
tag_list = ['JJ', 'JJR','JJS','NN','NNS','NNP','NNPS','VB','VBD','VBG','VBN','VBZ']
token_list_3 = []

for x in nltk.pos_tag(token_list_2):
    if x[1] in tag_list:
        if x[1] in ['JJ', 'JJR','JJS']:
            token_list_3.append(WordNetLemmatizer().lemmatize(x[0] , pos = 'a'))
        elif x[1] in ['NN','NNS','NNP','NNPS']:
            token_list_3.append(WordNetLemmatizer().lemmatize(x[0] , pos = 'n'))
        elif x[1] in  ['VB','VBD','VBG','VBN','VBZ']:
            token_list_3.append(WordNetLemmatizer().lemmatize(x[0] , pos = 'v'))

In [73]:
text_lem_list = []
text_lem_list.append(' '.join(token_list_3))

In [74]:
text_lem_list

['информации о этом вузе в интернете нет помимо официальной пора это исправить пишу со слов бывшего первокурсника пми о экзаменационных предметах алгебра дают и требуют матанализ вы его полюбите программирование за него сказать не могу будет новый препод во втором семестре появляется пара новых дискретная математика тяжёлый предмет дают его нудно но при желании его можно понять архитектура эвм благодаря этому предмету вы привыкните к самообразованию о неосновных предметах история семестр для того чтобы получить зачёт вам нужно будет подготовить минимум реферата и быть на парах преподаватель своеобразный.3 английский тут уж как повезёт если попадёте в сильную группу то ваши знания будут зависеть только от вас иначе ждёт куча домашки привет школа и желание учить english в универе отпадёт совсем русский семестр фразеололгизмов anki в помощь и будь на парах и зачёт в кармане физкультура пары в неделю если не пропускать то всё будет ок основы современной математики интересный но проходной п

In [77]:
!pip install transformers

You should consider upgrading via the 'C:\Users\Iljam\YandexDisk\ML_projects\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [78]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_checkpoint = 'cointegrated/rubert-tiny-sentiment-balanced'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
if torch.cuda.is_available():
    model.cuda()

def get_sentiment(text, return_type='label'):
    """ Calculate sentiment of a text. `return_type` can be 'label', 'score' or 'proba' """
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(model.device)
        proba = torch.sigmoid(model(**inputs).logits).cpu().numpy()[0]
    if return_type == 'label':
        return model.config.id2label[proba.argmax()]
    elif return_type == 'score':
        return proba.dot([-1, 0, 1])
    return proba

C:\Users\Iljam\YandexDisk\ML_projects\venv\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Iljam\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [79]:
sentiment_list = []
for text in df_uni['Отзыв']:
    sentiment_list.append(get_sentiment(text, 'score'))

In [81]:
len(sentiment_list)

1701

In [82]:
df_uni['Тональность'] = sentiment_list

In [83]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained('tinkoff-ai/response-toxicity-classifier-base')
model = AutoModelForSequenceClassification.from_pretrained('tinkoff-ai/response-toxicity-classifier-base')

In [84]:
sentiment_list = []
for text in tqdm(df_uni['Отзыв']):
    inputs = tokenizer(text, max_length=512, add_special_tokens=False, return_tensors='pt')
    with torch.inference_mode():
        logits = model(**inputs).logits
        probas = torch.softmax(logits, dim=-1)[0].cpu().detach().numpy()
        sentiment_list.append(probas)

  0%|          | 0/1701 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [85]:
len(sentiment_list)

1701

In [86]:
df_uni['Токсичность_1'] = sentiment_list

In [87]:
from transformers import BertForSequenceClassification

LABELS = ['neutral', 'happiness', 'sadness', 'enthusiasm', 'fear', 'anger', 'disgust']
tokenizer = AutoTokenizer.from_pretrained('Aniemore/rubert-tiny2-russian-emotion-detection')
model = BertForSequenceClassification.from_pretrained('Aniemore/rubert-tiny2-russian-emotion-detection')

@torch.no_grad()
def predict_emotion(text: str) -> str:
    """
        We take the input text, tokenize it, pass it through the model, and then return the predicted label
        :param text: The text to be classified
        :type text: str
        :return: The predicted emotion
    """
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.argmax(predicted, dim=1).numpy()

    return LABELS[predicted[0]]

@torch.no_grad()
def predict_emotions(text: str) -> list:
    """
        It takes a string of text, tokenizes it, feeds it to the model, and returns a dictionary of emotions and their
        probabilities
        :param text: The text you want to classify
        :type text: str
        :return: A dictionary of emotions and their probabilities.
    """
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    emotions_list = {}
    for i in range(len(predicted.numpy()[0].tolist())):
        emotions_list[LABELS[i]] = predicted.numpy()[0].tolist()[i]
    return emotions_list

In [88]:
sentiment_list = []
tqdm.pandas(desc='Processing Dataframe')
for text in tqdm(df_uni['Отзыв']):
    sentiment_list.append(predict_emotions(text))

  0%|          | 0/1701 [00:00<?, ?it/s]

In [89]:
sentiment_list

[{'neutral': 0.0632552057504654,
  'happiness': 0.007993282750248909,
  'sadness': 0.07387907058000565,
  'enthusiasm': 0.07502827048301697,
  'fear': 0.3409048914909363,
  'anger': 0.4268500804901123,
  'disgust': 0.01208913791924715},
 {'neutral': 0.05574873834848404,
  'happiness': 0.008815949782729149,
  'sadness': 0.055222101509571075,
  'enthusiasm': 0.0739891305565834,
  'fear': 0.4189966917037964,
  'anger': 0.3756764233112335,
  'disgust': 0.011550898663699627},
 {'neutral': 0.05574873834848404,
  'happiness': 0.008815949782729149,
  'sadness': 0.055222101509571075,
  'enthusiasm': 0.0739891305565834,
  'fear': 0.4189966917037964,
  'anger': 0.3756764233112335,
  'disgust': 0.011550898663699627},
 {'neutral': 0.0005717346793971956,
  'happiness': 0.9970797896385193,
  'sadness': 0.0003491798706818372,
  'enthusiasm': 0.0005371652077883482,
  'fear': 0.0006938545266166329,
  'anger': 0.0005797261255793273,
  'disgust': 0.00018845629529096186},
 {'neutral': 0.05574873834848404,


In [90]:
df_uni['Эмоции все'] = sentiment_list

In [91]:
df_uni

,Отзыв,День,Месяц,Год,Университет,Тональность,Токсичность_1,Эмоции все
0,\n\n\nИнформации о этом ВУЗе в интернете нет (...,21,8,2021,АГУ,0.873096,"[0.9921262, 0.00023803227, 0.0007774583, 0.006...","{'neutral': 0.0632552057504654, 'happiness': 0..."
1,\n\n\nИнформации о этом ВУЗе в интернете нет (...,12,2,2017,АлтГУ,0.873096,"[0.9903632, 0.00025323546, 0.00096872175, 0.00...","{'neutral': 0.05574873834848404, 'happiness': ..."
2,\n\n\nИнформации о этом ВУЗе в интернете нет (...,2,8,2022,Алфёровский университет,0.873096,"[0.9903632, 0.00025323546, 0.00096872175, 0.00...","{'neutral': 0.05574873834848404, 'happiness': ..."
3,\n\n\nЗдравствуйте! Меня зовут Александр. Я жи...,1,11,2016,Алфёровский университет,0.740849,"[0.99739707, 0.0004277167, 0.00027420663, 0.00...","{'neutral': 0.0005717346793971956, 'happiness'..."
4,\n\n\nИнформации о этом ВУЗе в интернете нет (...,8,6,2022,АмГМА,0.873096,"[0.9903632, 0.00025323546, 0.00096872175, 0.00...","{'neutral': 0.05574873834848404, 'happiness': ..."
...,...,...,...,...,...,...,...,...
1696,"\n\n\nЗа 5 лет повидала многого, но по порядку...",8,11,2017,ЮФУ,0.113793,"[0.97855365, 0.00027898338, 0.0017375285, 0.01...","{'neutral': 0.015182697214186192, 'happiness':..."
1697,\n\n\nИнформации о этом ВУЗе в интернете нет (...,23,6,2021,ЯГПУ им. Ушинского,0.873096,"[0.9903632, 0.00025323546, 0.00096872175, 0.00...","{'neutral': 0.05574873834848404, 'happiness': ..."
1698,\n\n\nЗдравствуйте! Меня зовут Александр. Я жи...,1,3,2021,ЯГПУ им. Ушинского,0.740849,"[0.99739707, 0.0004277167, 0.00027420663, 0.00...","{'neutral': 0.0005717346793971956, 'happiness'..."
1699,"\n\n\nС удивлением для себя обнаружил, что про...",16,2,2021,ЯГПУ им. Ушинского,0.556827,"[0.9961887, 0.0002449723, 0.00038222753, 0.003...","{'neutral': 0.005898027680814266, 'happiness':..."


In [92]:
df_uni.to_excel('Uni_with_emotions.xlsx')

In [3]:
df = pd.read_excel('Uni_with_emotions.xlsx')

In [4]:
df.head()

,Unnamed: 0,Отзыв,День,Месяц,Год,Университет,Тональность,Токсичность_1,Эмоции все
0,0,\n\n\nИнформации о этом ВУЗе в интернете нет (...,21,8,2021,АГУ,0.873096,[9.9212623e-01 2.3803227e-04 7.7745831e-04 6.8...,"{'neutral': 0.0632552057504654, 'happiness': 0..."
1,1,\n\n\nИнформации о этом ВУЗе в интернете нет (...,12,2,2017,АлтГУ,0.873096,[9.9036318e-01 2.5323546e-04 9.6872175e-04 8.4...,"{'neutral': 0.05574873834848404, 'happiness': ..."
2,2,\n\n\nИнформации о этом ВУЗе в интернете нет (...,2,8,2022,Алфёровский университет,0.873096,[9.9036318e-01 2.5323546e-04 9.6872175e-04 8.4...,"{'neutral': 0.05574873834848404, 'happiness': ..."
3,3,\n\n\nЗдравствуйте! Меня зовут Александр. Я жи...,1,11,2016,Алфёровский университет,0.740849,[9.9739707e-01 4.2771670e-04 2.7420663e-04 1.9...,"{'neutral': 0.0005717346793971956, 'happiness'..."
4,4,\n\n\nИнформации о этом ВУЗе в интернете нет (...,8,6,2022,АмГМА,0.873096,[9.9036318e-01 2.5323546e-04 9.6872175e-04 8.4...,"{'neutral': 0.05574873834848404, 'happiness': ..."


In [5]:
df = df.drop('Unnamed: 0', axis = 1)

In [31]:
list(ast.literal_eval(df.iloc[0, -1]).keys())

['neutral', 'happiness', 'sadness', 'enthusiasm', 'fear', 'anger', 'disgust']

In [26]:
neutral = []
happiness = []
sadness = []
enthisiasm = []
fear = []
anger = []
disgust = []

In [33]:
import ast
for i in range(len(df)):
    dic = ast.literal_eval(df.iloc[i, -1])
    keys_list = list(dic.keys())
    neutral.append(dic[keys_list[0]])
    happiness.append(dic[keys_list[1]])
    sadness.append(dic[keys_list[2]])
    enthisiasm.append(dic[keys_list[3]])
    fear.append(dic[keys_list[4]])
    anger.append(dic[keys_list[5]])
    disgust.append(dic[keys_list[6]])

In [34]:
df['Нейтральность'] =neutral
df['Счастье'] =happiness
df['Грусть'] =sadness
df['Энтузиазм'] =enthisiasm
df['Страх'] =fear
df['Гнев'] =anger
df['Омерзение'] =disgust

In [36]:
df = df.drop('Эмоции все', axis = 1)
df

,Отзыв,День,Месяц,Год,Университет,Тональность,Токсичность_1,Нейтральность,Счастье,Грусть,Энтузиазм,Страх,Гнев,Омерзение
0,\n\n\nИнформации о этом ВУЗе в интернете нет (...,21,8,2021,АГУ,0.873096,[9.9212623e-01 2.3803227e-04 7.7745831e-04 6.8...,0.063255,0.007993,0.073879,0.075028,0.340905,0.426850,0.012089
1,\n\n\nИнформации о этом ВУЗе в интернете нет (...,12,2,2017,АлтГУ,0.873096,[9.9036318e-01 2.5323546e-04 9.6872175e-04 8.4...,0.055749,0.008816,0.055222,0.073989,0.418997,0.375676,0.011551
2,\n\n\nИнформации о этом ВУЗе в интернете нет (...,2,8,2022,Алфёровский университет,0.873096,[9.9036318e-01 2.5323546e-04 9.6872175e-04 8.4...,0.055749,0.008816,0.055222,0.073989,0.418997,0.375676,0.011551
3,\n\n\nЗдравствуйте! Меня зовут Александр. Я жи...,1,11,2016,Алфёровский университет,0.740849,[9.9739707e-01 4.2771670e-04 2.7420663e-04 1.9...,0.000572,0.997080,0.000349,0.000537,0.000694,0.000580,0.000188
4,\n\n\nИнформации о этом ВУЗе в интернете нет (...,8,6,2022,АмГМА,0.873096,[9.9036318e-01 2.5323546e-04 9.6872175e-04 8.4...,0.055749,0.008816,0.055222,0.073989,0.418997,0.375676,0.011551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1696,"\n\n\nЗа 5 лет повидала многого, но по порядку...",8,11,2017,ЮФУ,0.113793,[9.7855365e-01 2.7898338e-04 1.7375285e-03 1.9...,0.015183,0.017380,0.103969,0.009312,0.176968,0.665495,0.011693
1697,\n\n\nИнформации о этом ВУЗе в интернете нет (...,23,6,2021,ЯГПУ им. Ушинского,0.873096,[9.9036318e-01 2.5323546e-04 9.6872175e-04 8.4...,0.055749,0.008816,0.055222,0.073989,0.418997,0.375676,0.011551
1698,\n\n\nЗдравствуйте! Меня зовут Александр. Я жи...,1,3,2021,ЯГПУ им. Ушинского,0.740849,[9.9739707e-01 4.2771670e-04 2.7420663e-04 1.9...,0.000572,0.997080,0.000349,0.000537,0.000694,0.000580,0.000188
1699,"\n\n\nС удивлением для себя обнаружил, что про...",16,2,2021,ЯГПУ им. Ушинского,0.556827,[9.9618870e-01 2.4497230e-04 3.8222753e-04 3.1...,0.005898,0.124570,0.002152,0.817556,0.026515,0.018281,0.005029


In [39]:
df['Токсичность_1'][0].split()

['[9.9212623e-01', '2.3803227e-04', '7.7745831e-04', '6.8582166e-03]']

In [40]:
ok_percent = []
toxic_percent = []
sever_toxic = []
risk_toxic = []

In [41]:
for i in df['Токсичность_1']:
    toxix_values = i.split()
    ok_percent.append(toxix_values[0])
    toxic_percent.append(toxix_values[1])
    sever_toxic.append(toxix_values[2])
    risk_toxic.append(toxix_values[3])

In [42]:
df['Токсичность в пределах нормы'] = ok_percent
df['Контекстная токсичность'] = toxic_percent
df['Серьезная токсичность'] = sever_toxic
df['Рискованная токсичность'] = risk_toxic

In [50]:
df['Контекстная токсичность'] = pd.to_numeric(df['Контекстная токсичность'])

In [51]:
df['Серьезная токсичность']= pd.to_numeric(df['Серьезная токсичность'])

In [62]:
for i, element in enumerate(df['Токсичность в пределах нормы']):
    df['Токсичность в пределах нормы'][i] = element[1:]

C:\Users\Iljam\AppData\Local\Temp\ipykernel_8104\3831276102.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Токсичность в пределах нормы'][i] = element[1:]


In [63]:
df['Токсичность в пределах нормы'] = pd.to_numeric(df['Токсичность в пределах нормы'])

In [64]:
df

,Отзыв,День,Месяц,Год,Университет,Тональность,Токсичность_1,Нейтральность,Счастье,Грусть,Энтузиазм,Страх,Гнев,Омерзение,Токсичность в пределах нормы,Контекстная токсичность,Серьезная токсичность,Рискованная токсичность
0,\n\n\nИнформации о этом ВУЗе в интернете нет (...,21,8,2021,АГУ,0.873096,[9.9212623e-01 2.3803227e-04 7.7745831e-04 6.8...,0.063255,0.007993,0.073879,0.075028,0.340905,0.426850,0.012089,0.992126,0.000238,0.000777,6.8582166e-03]
1,\n\n\nИнформации о этом ВУЗе в интернете нет (...,12,2,2017,АлтГУ,0.873096,[9.9036318e-01 2.5323546e-04 9.6872175e-04 8.4...,0.055749,0.008816,0.055222,0.073989,0.418997,0.375676,0.011551,0.990363,0.000253,0.000969,8.4148953e-03]
2,\n\n\nИнформации о этом ВУЗе в интернете нет (...,2,8,2022,Алфёровский университет,0.873096,[9.9036318e-01 2.5323546e-04 9.6872175e-04 8.4...,0.055749,0.008816,0.055222,0.073989,0.418997,0.375676,0.011551,0.990363,0.000253,0.000969,8.4148953e-03]
3,\n\n\nЗдравствуйте! Меня зовут Александр. Я жи...,1,11,2016,Алфёровский университет,0.740849,[9.9739707e-01 4.2771670e-04 2.7420663e-04 1.9...,0.000572,0.997080,0.000349,0.000537,0.000694,0.000580,0.000188,0.997397,0.000428,0.000274,1.9010528e-03]
4,\n\n\nИнформации о этом ВУЗе в интернете нет (...,8,6,2022,АмГМА,0.873096,[9.9036318e-01 2.5323546e-04 9.6872175e-04 8.4...,0.055749,0.008816,0.055222,0.073989,0.418997,0.375676,0.011551,0.990363,0.000253,0.000969,8.4148953e-03]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1696,"\n\n\nЗа 5 лет повидала многого, но по порядку...",8,11,2017,ЮФУ,0.113793,[9.7855365e-01 2.7898338e-04 1.7375285e-03 1.9...,0.015183,0.017380,0.103969,0.009312,0.176968,0.665495,0.011693,0.978554,0.000279,0.001738,1.9429790e-02]
1697,\n\n\nИнформации о этом ВУЗе в интернете нет (...,23,6,2021,ЯГПУ им. Ушинского,0.873096,[9.9036318e-01 2.5323546e-04 9.6872175e-04 8.4...,0.055749,0.008816,0.055222,0.073989,0.418997,0.375676,0.011551,0.990363,0.000253,0.000969,8.4148953e-03]
1698,\n\n\nЗдравствуйте! Меня зовут Александр. Я жи...,1,3,2021,ЯГПУ им. Ушинского,0.740849,[9.9739707e-01 4.2771670e-04 2.7420663e-04 1.9...,0.000572,0.997080,0.000349,0.000537,0.000694,0.000580,0.000188,0.997397,0.000428,0.000274,1.9010528e-03]
1699,"\n\n\nС удивлением для себя обнаружил, что про...",16,2,2021,ЯГПУ им. Ушинского,0.556827,[9.9618870e-01 2.4497230e-04 3.8222753e-04 3.1...,0.005898,0.124570,0.002152,0.817556,0.026515,0.018281,0.005029,0.996189,0.000245,0.000382,3.1840086e-03]


In [65]:
for i, element in enumerate(df['Рискованная токсичность']):
    df['Рискованная токсичность'][i] = element[:-1]

C:\Users\Iljam\AppData\Local\Temp\ipykernel_8104\3957142037.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Рискованная токсичность'][i] = element[:-1]


In [68]:
df = df.drop('Токсичность_1', axis = 1)

In [69]:
df

,Отзыв,День,Месяц,Год,Университет,Тональность,Нейтральность,Счастье,Грусть,Энтузиазм,Страх,Гнев,Омерзение,Токсичность в пределах нормы,Контекстная токсичность,Серьезная токсичность,Рискованная токсичность
0,\n\n\nИнформации о этом ВУЗе в интернете нет (...,21,8,2021,АГУ,0.873096,0.063255,0.007993,0.073879,0.075028,0.340905,0.426850,0.012089,0.992126,0.000238,0.000777,6.8582166e-03
1,\n\n\nИнформации о этом ВУЗе в интернете нет (...,12,2,2017,АлтГУ,0.873096,0.055749,0.008816,0.055222,0.073989,0.418997,0.375676,0.011551,0.990363,0.000253,0.000969,8.4148953e-03
2,\n\n\nИнформации о этом ВУЗе в интернете нет (...,2,8,2022,Алфёровский университет,0.873096,0.055749,0.008816,0.055222,0.073989,0.418997,0.375676,0.011551,0.990363,0.000253,0.000969,8.4148953e-03
3,\n\n\nЗдравствуйте! Меня зовут Александр. Я жи...,1,11,2016,Алфёровский университет,0.740849,0.000572,0.997080,0.000349,0.000537,0.000694,0.000580,0.000188,0.997397,0.000428,0.000274,1.9010528e-03
4,\n\n\nИнформации о этом ВУЗе в интернете нет (...,8,6,2022,АмГМА,0.873096,0.055749,0.008816,0.055222,0.073989,0.418997,0.375676,0.011551,0.990363,0.000253,0.000969,8.4148953e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1696,"\n\n\nЗа 5 лет повидала многого, но по порядку...",8,11,2017,ЮФУ,0.113793,0.015183,0.017380,0.103969,0.009312,0.176968,0.665495,0.011693,0.978554,0.000279,0.001738,1.9429790e-02
1697,\n\n\nИнформации о этом ВУЗе в интернете нет (...,23,6,2021,ЯГПУ им. Ушинского,0.873096,0.055749,0.008816,0.055222,0.073989,0.418997,0.375676,0.011551,0.990363,0.000253,0.000969,8.4148953e-03
1698,\n\n\nЗдравствуйте! Меня зовут Александр. Я жи...,1,3,2021,ЯГПУ им. Ушинского,0.740849,0.000572,0.997080,0.000349,0.000537,0.000694,0.000580,0.000188,0.997397,0.000428,0.000274,1.9010528e-03
1699,"\n\n\nС удивлением для себя обнаружил, что про...",16,2,2021,ЯГПУ им. Ушинского,0.556827,0.005898,0.124570,0.002152,0.817556,0.026515,0.018281,0.005029,0.996189,0.000245,0.000382,3.1840086e-03


In [70]:
df.to_excel('totally_ready_df_uni.xlsx')